In [92]:
from collections import defaultdict

transactions = []
min_util = 50
utility_lists = {}
parsed_trans = []
TWU = defaultdict(int)
revised_trans = []
with open('test.txt', 'r') as file:
    for line in file:
        transactions.append(line.strip())

In [94]:
def parse(line, tid):
    items, total_util, item_utils = line.split(':')
    items = list(map(int, items.strip().split()))
    total_util = float(total_util.strip())
    item_utils = list(map(float, item_utils.strip().split()))
    parsed_trans.append((tid, items, item_utils, total_util))
    for item in items:
        TWU[item] += total_util


In [96]:
def revise():
    kept = {item for item in TWU if TWU[item] >= min_util}
    sorted_items = sorted(kept, key=lambda i: (TWU[i], i))
    ordering = {item: idx for idx, item in enumerate(sorted_items)}

    for tid, items, utils, tu in parsed_trans:
        revised = [(item, util) for item, util in zip(items, utils) if item in kept]
        revised.sort(key=lambda x: ordering[x[0]])
        if revised:
            revised_trans.append((tid, revised))

In [98]:
class UtilityListEntry:
    def __init__(self, tid, iu, ru):
        self.tid = tid
        self.iu = iu
        self.ru = ru

In [100]:
def build_item_UL():
    for tid, revised in revised_trans:
        items = [item for item,_ in revised]
        utils = [util for _,util in revised]
        for i in range(len(items)):
            item = items[i]
            iu = utils[i]
            ru = sum(utils[i+1:])
            if item not in utility_lists:
                utility_lists[item] = []
            utility_lists[item].append(UtilityListEntry(tid, iu, ru))

In [102]:
def construct(prefix, x, y):
    newUL = []
    mapY = {e.tid: e for e in y}
    mapP = {e.tid: e for e in prefix} if prefix else {}

    for Xentry in x:
        Yentry = mapY.get(Xentry.tid)
        if Yentry:
            if prefix:
                Pentry = mapP[Xentry.tid]
                iu = Xentry.iu + Yentry.iu - Pentry.iu
            else:
                iu = Xentry.iu + Yentry.iu
            ru = Yentry.ru
            newUL.append(UtilityListEntry(tid, iu, ru))

    return newUL
    

In [104]:
def huiMiner(prefix, ULs):
    for i in range(len(ULs)):
        Xi, xUL = ULs[i]
        newPrefix = prefix + (Xi,)
        sumIU = sum(e.iu for e in xUL)
        sumTOT = sum(e.ru + e.iu for e in xUL)

        if sumIU >= min_util:
            print(f"High Utility Itemset: {newPrefix}, Utility: {sumIU}")

        if sumTOT >= min_util:
            extendedULs = []
            for j in range(i+1, len(ULs)):
                Yj, yUL = ULs[j]
                newUL = construct(xUL if not prefix else utility_lists[prefix], xUL, yUL)
                if newUL:
                    extendedULs.append((Yj, newUL))
            huiMiner(newPrefix, extendedULs)
            
    

In [108]:
for tid, line in enumerate(transactions):
    parse(line, tid)

revise()
build_item_UL()
ULs = sorted([(item, utility_lists[item]) for item in utility_lists], key=lambda x: revised_trans[x[0]])
HUIMiner(tuple(), ULs)

IndexError: list index out of range